## Import Required Libraries

Let's get started by importing any required libraries.

dbLite is a small library that will wick away some of the boilerplate code involved in writing queries against Postgres.

In [1]:
import pandas as pd
from common.dbLite import dbLite

db = dbLite()

## Set Up the Database

Run this once only (or if you wish you re-set the state of the database).

In [4]:
# Set up our tables

!python3 create_tables.py

## Example 1: Using Pandas's read_sql method

You can use any of the Pandas sql methods by passing the connection object, like so:

In [2]:
pd.read_sql('select count(*) from projects', db.connect())

## Example 2: Using dbLite

To use dbLite, simply reference the db object we created earlier, and call the sql method.

In [3]:
# Creating a table, then selecting from it.
sql = """
    DROP TABLE IF EXISTS invited_users_by_project;
    CREATE TABLE invited_users_by_project AS
    SELECT
        t1.project_gid,
        count(distinct t2.user_gid) as invited_users
    FROM projects t1
    INNER JOIN activity t2
    ON t1.project_gid = t2.project_gid
    GROUP BY 1
    ORDER BY 2 DESC;
"""
db.sql(sql)
sql = """
    SELECT *
    FROM invited_users_by_project;
"""
df1 = db.sql(sql)
df1

In [9]:
# Creating a table, then selecting from it.
sql = """
    DROP VIEW IF EXISTS top_5_invited_users;
    CREATE VIEW top_5_invited_users AS
    SELECT
        t1.user_gid,
        count(distinct t2.project_gid) as invited_projects
    FROM users t1
    INNER JOIN activity t2
    ON t1.user_gid = t2.user_gid
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5;
"""
db.sql(sql)
sql = """
    SELECT *
    FROM top_5_invited_users;
"""
df2 = db.sql(sql)
df2

In [10]:
# Example of using a parameterized query
sql = "SELECT COUNT(*) FROM activity WHERE project_gid = %(project_gid)s"
params = {
    'project_gid': 1,
}
db.sql(sql,params)